In [10]:
library(repr)
library(tidyverse)
library(tidymodels)
library(readr)
library(janitor)

#### Group project proposal

# Data Analysis on Air Quality Index of Counties in the US

DSCI100 group porject-006-14  
Group member:  
`Dorthy GER`  
`Charlie Hua`  
`Callie Phelps`  
`Hancheng Zhang 29643160`


## Introduction

## Preliminary exploratory data analysis

This data analysis project is based on the "annual_aqi_by_county_2021.csv" dataset. It can be directly read from the web as follow.

In [6]:
url <- "https://raw.githubusercontent.com/Hansen0014/Air-quality-data/main/annual_aqi_by_county_2021.csv"
Air_quality <- read.csv(url)
head(Air_quality)

,State,County,Year,Days.with.AQI,Good.Days,Moderate.Days,Unhealthy.for.Sensitive.Groups.Days,Unhealthy.Days,Very.Unhealthy.Days,Hazardous.Days,Max.AQI,X90th.Percentile.AQI,Median.AQI,Days.CO,Days.NO2,Days.Ozone,Days.SO2,Days.PM2.5,Days.PM10
,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,Alabama,Baldwin,2021,203,190,13,0,0,0,0,61,49,37,0,0,166,0,37,0
2,Alabama,Clay,2021,73,66,7,0,0,0,0,67,50,28,0,0,0,0,73,0
3,Alabama,DeKalb,2021,239,211,28,0,0,0,0,84,51,38,0,0,213,0,26,0
4,Alabama,Elmore,2021,180,177,3,0,0,0,0,87,45,34,0,0,180,0,0,0
5,Alabama,Etowah,2021,204,178,26,0,0,0,0,77,54,38,0,0,158,0,46,0
6,Alabama,Jefferson,2021,182,90,90,1,0,1,0,213,74,51,2,1,65,0,113,1


Since we want to predict does a county face existing/potential air quality issues. We decide to set a bar towards `90th_Percentile_AQI`. Counties with 90th percentile AQI higher than or equal to 100 would be defined as having existing/potential air quality, and counties with 90th percentile AQI lower than 100 would be defined as having no air quality issue.

In order to achive this goal, we need to wangle 

In [13]:
Air_quality_c <- clean_names(Air_quality) %>%
        



state,county,year,days_with_aqi,good_days,moderate_days,unhealthy_for_sensitive_groups_days,unhealthy_days,very_unhealthy_days,hazardous_days,max_aqi,x90th_percentile_aqi,median_aqi,days_co,days_no2,days_ozone,days_so2,days_pm2_5,days_pm10
<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
Alabama,Baldwin,2021,203,190,13,0,0,0,0,61,49,37,0,0,166,0,37,0
Alabama,Clay,2021,73,66,7,0,0,0,0,67,50,28,0,0,0,0,73,0
Alabama,DeKalb,2021,239,211,28,0,0,0,0,84,51,38,0,0,213,0,26,0
Alabama,Elmore,2021,180,177,3,0,0,0,0,87,45,34,0,0,180,0,0,0
Alabama,Etowah,2021,204,178,26,0,0,0,0,77,54,38,0,0,158,0,46,0
Alabama,Jefferson,2021,182,90,90,1,0,1,0,213,74,51,2,1,65,0,113,1
Alabama,Madison,2021,173,139,33,1,0,0,0,101,56,42,0,0,79,0,81,13
Alabama,Mobile,2021,243,162,81,0,0,0,0,81,58,46,0,0,48,0,195,0
Alabama,Montgomery,2021,232,182,50,0,0,0,0,93,57,40,0,0,108,0,124,0
